In [20]:
import keras
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, Flatten
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

In [2]:
# load_data returns a tupple containing the tringing data in 0 and test data in 1
(X_train_full, y_train_full), (X_test, y_test) = trainfashion_mnist = fashion_mnist.load_data()

In [3]:
print(X_train_full.shape, X_train_full.dtype)
print(y_train_full.shape, y_train_full.dtype)

(60000, 28, 28) uint8
(60000,) uint8


In [4]:
X_test.shape

(10000, 28, 28)

In [5]:
y_train_full.shape

(60000,)

In [6]:
y_test.shape

(10000,)

In [7]:
type(y_test[0])

numpy.uint8

In [8]:
type(X_train_full[0])

numpy.ndarray

In [9]:

X_train, X_val = X_train_full[5000:] / 255, X_train_full[:5000] / 255
y_train, y_val = y_train_full[5000:], y_train_full[:5000]

In [10]:
model = Sequential()

model.add(Flatten(input_shape=[28, 28]))
model.add(Dense(300, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(10, activation='softmax'))




In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [13]:
model.fit(X_train, y_train, epochs=30, validation_data=(X_val, y_val))

Epoch 1/30
1719/1719 [==============================] - 6s 3ms/step - loss: 0.7006 - accuracy: 0.7691 - val_loss: 0.5111 - val_accuracy: 0.8222
Epoch 2/30
1719/1719 [==============================] - 4s 2ms/step - loss: 0.4834 - accuracy: 0.8306 - val_loss: 0.4469 - val_accuracy: 0.8488
Epoch 3/30
1719/1719 [==============================] - 4s 2ms/step - loss: 0.4407 - accuracy: 0.8446 - val_loss: 0.4213 - val_accuracy: 0.8564
Epoch 4/30
1719/1719 [==============================] - 4s 2ms/step - loss: 0.4135 - accuracy: 0.8539 - val_loss: 0.3930 - val_accuracy: 0.8676
Epoch 5/30
1719/1719 [==============================] - 4s 2ms/step - loss: 0.3934 - accuracy: 0.8606 - val_loss: 0.4007 - val_accuracy: 0.8552
Epoch 6/30
1719/1719 [==============================] - 4s 2ms/step - loss: 0.3773 - accuracy: 0.8668 - val_loss: 0.3699 - val_accuracy: 0.8708
Epoch 7/30
1719/1719 [==============================] - 4s 2ms/step - loss: 0.3630 - accuracy: 0.8708 - val_loss: 0.3605 - val_accuracy:

In [22]:
y_pred = np.argmax(model.predict(X_test), axis=1)
y_pred[0:5]

313/313 [==============================] - 1s 3ms/step


array([9, 2, 1, 1, 6], dtype=int64)

In [23]:
y_test[0:5]

array([9, 2, 1, 1, 6], dtype=uint8)

In [24]:
model.evaluate(X_test, y_test)

  1/313 [..............................] - ETA: 9s - loss: 66.5981 - accuracy: 0.8438

313/313 [==============================] - 1s 2ms/step - loss: 61.3343 - accuracy: 0.8553


[61.33428192138672, 0.8553000092506409]